## CX4240 project

## Classification of Acute Lymphoblastic Leukemia (ALL) in Blood Cell Images Using Machine Learning

# Feature Extractions


In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.stats as stat
import sklearn.preprocessing as pre
import glob
import mahotas as mt
import pywt
import seaborn as sns
import pandas as pd
import csv 
import matplotlib as mpl

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
%matplotlib inline


# Load Images

In [2]:
def find_path(stage, fold=None, label=None):
    """
    ARGS:    stage: 'train' or 'test'
             fold: int
             label: 'all' or 'hem'
    returns: general file path for given inputs
    """
    og_path = '/Users/1000j/CX4240+/Cell_Images/'
    if stage == 'train':
        return og_path + 'C-NMC_training_data/fold_' + str(fold) + '/' + label + '/*.bmp'
    elif stage == 'test':
        return og_path + 'C-NMC_test_prelim_phase_data/'*2 + '*.bmp'
    else:
        print('must enter "train" or "test"')
        return

def read_images(stage, fold=None, label=None):
    """
    ARGS:    stage: 'train' or 'test'
             fold: int
             label: 'all' or 'hem'
    returns: color: list of all color images
             gray: list of all gray images
             ids: list of associated image ids
    """
    
    img_path = find_path(stage, fold, label)
    start = len(img_path) - 5
    color, gray, ids = [], [], []
    ids = []
    for name in glob.glob(img_path):
        ids.append(name[start : -4])
        color.append(cv2.imread(name))
        gray.append(cv2.imread(name,0))
    color = np.array(color)
    gray = np.array(gray)
    ret, binary = cv2.threshold(gray,0,255,cv2.THRESH_BINARY)
    if stage=='train':
        print('done loading training %s images in fold%s' % (label,str(fold)))
    else:
        print('done loading test images')
    return color, gray, binary, ids

In [ ]:
# Load train images
ALL_0_color, ALL_0_gray, ALL_0_binary, ALL_0_ids = read_images('train', 0, 'all')
hem_0_color, hem_0_gray, hem_0_binary, hem_0_ids = read_images('train', 0, 'hem')
ALL_1_color, ALL_1_gray, ALL_1_binary, ALL_1_ids = read_images('train', 1, 'all')
hem_1_color, hem_1_gray, hem_1_binary, hem_1_ids = read_images('train', 1, 'hem')
ALL_2_color, ALL_2_gray, ALL_2_binary, ALL_2_ids = read_images('train', 2, 'all')
hem_2_color, hem_2_gray, hem_2_binary, hem_2_ids = read_images('train', 2, 'hem')
ALL_0_color = np.concatenate((ALL_0_color,ALL_1_color,ALL_2_color), axis=0)
ALL_0_gray = np.concatenate((ALL_0_gray,ALL_1_gray,ALL_2_gray), axis=0)
ALL_0_binary = np.concatenate((ALL_0_binary,ALL_1_binary,ALL_2_binary), axis=0)
hem_0_color = np.concatenate((hem_0_color,hem_1_color,hem_2_color), axis=0)
hem_0_gray = np.concatenate((hem_0_gray,hem_1_gray,hem_2_gray), axis=0)
hem_0_binary = np.concatenate((hem_0_binary,hem_1_binary,hem_2_binary), axis=0)


done loading training all images in fold0
done loading training hem images in fold0


In [ ]:
# Load test data
t_color, t_gray, t_binary, test_ids = read_images('test', 0)
test_ids = np.asarray(test_ids, dtype=int)
test_color = np.zeros(np.shape(t_color), dtype=np.uint8)
test_gray = np.zeros(np.shape(t_gray), dtype=np.uint8)
test_binary = np.zeros(np.shape(t_binary), dtype=np.uint8)
test_color[test_ids-1] = t_color
test_gray[test_ids-1] = t_gray
test_binary[test_ids-1] = t_binary

In [ ]:
# number of images
n_ALL = len(ALL_0_color)
n_hem = len(hem_0_color)
n_test = len(test_color)
# Shape of images
print(np.shape(ALL_0_color))
print(np.shape(hem_0_color))
print(np.shape(test_color))

In [ ]:
plt.figure(num=None, figsize=(10, 8), dpi=160, facecolor='w', edgecolor='k')
plt.subplot(2,3,1), plt.imshow(ALL_0_color[172]), plt.title('ALL color')
plt.subplot(2,3,2), plt.imshow(ALL_0_gray[172],'gray'), plt.title('ALL grayscale')
plt.subplot(2,3,3), plt.imshow(ALL_0_binary[172],'gray'), plt.title('ALL binary')
plt.subplot(2,3,4), plt.imshow(hem_0_color[127]), plt.title('Normal color')
plt.subplot(2,3,5), plt.imshow(hem_0_gray[127],'gray'), plt.title('Normal grayscale')
plt.subplot(2,3,6), plt.imshow(hem_0_binary[127],'gray'), plt.title('Normal binary')

In [ ]:
plt.figure(figsize = (20, 4))
for i, j in enumerate(np.random.randint(n_ALL, size = 5)):
    plt.subplot(1, 5, i+1)
    plt.imshow(ALL_0_color[j])
plt.suptitle('ALL Cells', fontsize=25)

plt.figure(figsize = (20, 4))
for i, j in enumerate(np.random.randint(n_hem, size = 5)):
    plt.subplot(1, 5, i+1)
    plt.imshow(hem_0_color[j])
plt.suptitle('Normal Cells', fontsize=25)


# Feature extraction
    F1: cell size
    F2: perimeter
    F3: form factor
    F4: roundness
    F5: length/diameter ratio
    F6: compactness
    F7-F9: contour signature of nucleus: 
        variance, skewness, and kurtosis of the distances between centroid and contour points
    F10-F22: Haralick texture 
        angular second moment
        contrast
        correlation
        variance
        inverse difference moment
        sum average
        sum variance
        sum entropy
        difference entropy
        information measures of correlation (F12, F13)
    F23-30: Haar wavelet texture
        Means and variances of low-pass filtered appriximation image 
        and high-pass filterd in horizontal, vertical, and diagonal directions
    F31-F36: color features
        Means in RGB
        Means in HSV



In [ ]:
features = ['Cell Size','Perimeter','Form Factor','Roundness','Length/Diameter Ratio','Compactness',
            'Boundary Roughness Variance','Boundary Roughness Skewness','Boundary Roughness Kurtosis',
            'Haralick Angular Second Moment','Haralick Contrast','Haralick Correlation','Haralick Variance',
            'Haralick Inverse Difference Moment','Haralick Sum Average','Haralick Sum Variance',
            'Haralick Sum Entropy','Haralick Entropy','Haralick Difference Variance','Haralick Difference Entropy',
            'Haralick Information Measures of Correlation 1','Haralick Information Measures of Correlation 2',
            'Wavelet Approximation Mean','Wavelet Horizontal Mean','Wavelet Vertical Mean','Wavelet Diagonal Mean',
            'Wavelet Approximation Variance', 'Wavelet Horizontal Variance','Wavelet Vertical Variance',
            'Wavelet Diagonal Variance','Red Mean','Green Mean','Blue Mean','Hue Mean','Saturation Mean','Value Mean',
            'Intensity Mean','Intensity Variance']

## Morphological feature extraction

In [ ]:
# Morphological features
def get_cell_size(image):
    """
    ARGS:    
        image: black and white image
    returns:
        size: the number of pixels in the cell
    """
#    ret,thresh=cv2.threshold(image,THRESH,255,cv2.THRESH_BINARY)
#    return cv2.countNonZero(image)
    return np.count_nonzero(image, axis=(1,2))

def get_cell_perimeter(image):
    """
    ARGS:    
        image: black and white image
    returns:
        perimeter: the number of pixels in the perimeter
        cnt: x,y coordiantes of boundary (list)
    """
#    ret,thresh = cv2.threshold(image,THRESH,255,cv2.THRESH_BINARY)
    perimeter = np.zeros(len(image))
    cnt = []
    for i in range(len(image)):
        _, contour, _ = cv2.findContours(image[i], cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        perimeter[i] = contour[0].shape[0]
        cnt.append(contour[0])
    return perimeter, cnt

def get_centeroid(image):
    """
    arg:
        image: binary image
    returns:
        cx: x coordinate of center
        cy: y coordinate of center
    """
    cx = np.zeros(len(image), dtype=int)
    cy = np.zeros(len(image), dtype=int)
    for i in range(len(image)):
        M = cv2.moments(image[i])
        cx[i] = int(M['m10']/M['m00'])
        cy[i] = int(M['m01']/M['m00'])
    return cx,cy
    
def get_diameter(image):
    """
    args:
        image: binary images
    returns: 
        major_d: the maximum diameter (number of pixels) on the major axis
        minor_d: the diameter (number of pixels) on the minor axis (perpendicular to the major axis)
        min_d: the minimum diameter 
    """
    (rows, cols) = image[0].shape[:2] 
    major_d = np.zeros(len(image))
    minor_d = np.zeros(len(image))
    min_d = rows*np.ones(len(image))
    major_img = np.zeros(np.shape(image))
    minor_img = np.zeros(np.shape(image))
    for i in range(len(image)):
        for ang in range(0,180,10):
            # Images are rotated 0 to 170 degrees with the increment of 10 degrees
            # getRotationMatrix2D creates a matrix needed for transformation. 
            M = cv2.getRotationMatrix2D((cols / 2, rows / 2), ang, 1) 
            rotated_img = cv2.warpAffine(image[i], M, (cols, rows))
            temp_d = np.count_nonzero(rotated_img[int(rows/2),:]) # count pixels
            if temp_d > major_d[i]:
                major_d[i] = temp_d
                minor_d[i] = np.count_nonzero(rotated_img[:,int(cols/2)]) 
            if temp_d < min_d[i]:
                min_d[i] = temp_d
    return major_d, minor_d, min_d

In [ ]:
#F1 size
F1_all = get_cell_size(ALL_0_binary)
F1_hem = get_cell_size(hem_0_binary)
F1_t = get_cell_size(test_binary)
#F2 perimeter
F2_all, ALL_0_cnt = get_cell_perimeter(ALL_0_binary)
F2_hem, hem_0_cnt = get_cell_perimeter(hem_0_binary)
F2_t, test_cnt = get_cell_perimeter(test_binary)

ALL_0_major_d, ALL_0_minor_d, ALL_0_min_d = get_diameter(ALL_0_binary)
hem_0_major_d, hem_0_minor_d, hem_0_min_d = get_diameter(hem_0_binary)
test_major_d, test_minor_d, test_min_d = get_diameter(test_binary)
ALL_0_cx, ALL_0_cy = get_centeroid(ALL_0_binary)
hem_0_cx, hem_0_cy = get_centeroid(hem_0_binary)
test_cx, test_cy = get_centeroid(test_binary)

In [ ]:
def get_form_factor(area, perimeter):
    """
    args: 
        area: the area of cell (cell size)
        perimeter: the total number of pixels representing the cell boundary
    returns: 
        form_factor: shape of cell
    """
    return 4*np.pi*area/perimeter**2

def get_roundness(area, major_diameter):
    """
    args: 
        area: the area of cell (cell size)
        major_diameter: the diameter of cell on the major axis
    returns: 
        roundness: the degree to which the cell shape differs from that of a circle
    """
    return 4*area/(np.pi*major_diameter**2)

def get_diameter_ratio(major_d, minor_d):
    """
    args:
        major_d: the maximum diameter (number of pixels) on the major axis
        minor_d: the diameter (number of pixels) on the minor axis (perpendicular to the major axis
    returns: 
        ratio: ratio of major_d and minor_d
    """
    return major_d/minor_d

def get_compactness(area, major_diameter):
    """
    args: 
        area: area of cell (cell size)
        major_diameter: the diameter of cell on the major axis
    returns: 
        compactness: the degree to how compact the shape of cell is
    """
    return ((4*area/np.pi)**(1/2))/major_diameter

def get_contour_signature(image, cnt, cx, cy):
    """ Boundary roughness
    arg:
        image: binary images
        cnt: (list of numpy array ixjxk) x,y coordiantes of boundary
        cx: x coordinate of center
        cy: y coordinage of center
    returns:
        variance, skewness, kurtosis
    """
    variance = np.zeros(len(image))
    skewness = np.zeros(len(image))
    kurtosis = np.zeros(len(image))
    for i in range(len(image)):
        xy = cnt[i]
        norm2 = np.zeros(len(xy))
        for j in range(len(xy)):
            norm2[j] = np.linalg.norm((cx[i],cy[i])-xy[j,0,:])
        variance[i] = np.var(norm2)
        skewness[i] = stat.skew(norm2)
        kurtosis[i] = stat.kurtosis(norm2)
    return variance, skewness, kurtosis
        

In [ ]:
#F3 form factor
F3_all = get_form_factor(F1_all, F2_all)
F3_hem = get_form_factor(F1_hem, F2_hem)
F3_t = get_form_factor(F1_t, F2_t)
#F4 roundness
F4_all = get_roundness(F1_all, ALL_0_major_d)
F4_hem = get_roundness(F1_hem, hem_0_major_d)
F4_t = get_roundness(F1_t, test_major_d)
#F5 diameter ratio
F5_all = get_diameter_ratio(ALL_0_major_d, ALL_0_minor_d)
F5_hem = get_diameter_ratio(hem_0_major_d, hem_0_minor_d)
F5_t = get_diameter_ratio(test_major_d, test_minor_d)
#F6 compactness
F6_all = get_compactness(F1_all, ALL_0_major_d)
F6_hem = get_compactness(F1_hem, hem_0_major_d)
F6_t = get_compactness(F1_t, test_major_d)
#F7-F9 variance, skewness, kurtosis (boundary roughness)
F7_all, F8_all, F9_all = get_contour_signature(ALL_0_binary, ALL_0_cnt, ALL_0_cx, ALL_0_cy)
F7_hem, F8_hem, F9_hem = get_contour_signature(hem_0_binary, hem_0_cnt, hem_0_cx, hem_0_cy)
F7_t, F8_t, F9_t = get_contour_signature(test_binary, test_cnt, test_cx, test_cy)

## Texture features extraction

In [ ]:
# Texture features
def crop_image(image, cx, cy, min_d=50):
    """
        args:
            image: grayscale image
            cx: x coordinate of center (row)
            cy: y coordiante of center (column)
            mid_d: (int) minimum diameter of all cells
        returns:
            new_img: cropped images with size min_d by min_d
    """
    rows, cols = image[0].shape[:2] 
    l = np.int(np.min(min_d)/2)
    for i in range(len(image)):
        new_img = image[:,cx[i]-l:cx[i]+l,cy[i]-l:cy[i]+l]
    return new_img

def Haralick(img):
    '''Haralick texture using Gray-level co-occurrence matrix (GLCM)
    arg: 
        img: N number of MxM grayscale images
    return: 
        Haralick_texture: Nx13 array with 13 features 
    '''
    Haralick_texture = []
    for i in range(len(img)):
        feat = mt.features.haralick(img[i,:,:], return_mean=True, ignore_zeros=True)
        Haralick_texture.append(feat)
    Haralick_texture = np.array(Haralick_texture)     
    return Haralick_texture
    
def Haar_wavelet(img):
    '''Extract Haar wavelet texture features
    Means and variances of low-pass filtered appriximation image 
    and high-pass filterd in horizontal, vertical, and diagonal directions
    arg:
        img: N number of MxM grayscale images
    return: 
        Haar_wavelet_features: Nx8 array with 8 features
            (cA_mean, cH_mean, cV_mean, cD_mean, cA_var, cH_var, cV_var, and cD_var)
    '''
    cA, (cH, cV, cD) = pywt.dwt2(img,'haar')
    n = len(cA)
    cA_mean = np.reshape(cA.mean(axis=(1,2)), (n,1))
    cH_mean = np.reshape(cH.mean(axis=(1,2)), (n,1))
    cV_mean = np.reshape(cV.mean(axis=(1,2)), (n,1))
    cD_mean = np.reshape(cD.mean(axis=(1,2)), (n,1))
    cA_var = np.reshape(cA.var(axis=(1,2)), (n,1))
    cH_var = np.reshape(cH.var(axis=(1,2)), (n,1))
    cV_var = np.reshape(cV.var(axis=(1,2)), (n,1))
    cD_var = np.reshape(cD.var(axis=(1,2)), (n,1))
    
    Haar_wavelet_features = np.hstack((cA_mean,cH_mean,cV_mean,cD_mean,cA_var,cH_var,cV_var,cD_var))
    return Haar_wavelet_features

In [ ]:
#new images for texture feature extraction
ALL_0_texture = crop_image(ALL_0_gray, ALL_0_cx, ALL_0_cy)
hem_0_texture = crop_image(hem_0_gray, hem_0_cx, hem_0_cy)
test_texture = crop_image(test_gray, test_cx, test_cy)

#F10-F22
Haralick_all = Haralick(ALL_0_texture)
Haralick_hem = Haralick(hem_0_texture)
F10_F22_t = Haralick(test_texture)
#F23-30
Haar_wavelet_all = Haar_wavelet(ALL_0_texture)
Haar_wavelet_hem = Haar_wavelet(hem_0_texture)
F23_F30_t = Haar_wavelet(test_texture)

## Color feature extraction

In [ ]:
# Color features
def image_RGB_mean(image):
    """
    Args:
        image in RGB
    Return:
        A tuple of (Red, Green, Blue) with mean value among all non-black pixels    
    """
    flattened = image.reshape(-1, image.shape[-1])
    RED, GREEN, BLUE = (0,1,2)
    red_components = flattened[:, RED]
    green_components = flattened[:, GREEN]
    blue_components = flattened[:, BLUE]
    mask = (red_components != 0) | (green_components != 0) | (blue_components != 0)
    RGB_mean = np.mean(flattened[mask, :], axis = 0)
    red, green, blue = RGB_mean[0], RGB_mean[1], RGB_mean[2]
    return red, green, blue

def image_HSV(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

def image_HSV_mean(image):
    """
    Args:
        image in HSV
    Return:
        A tuple of (Hue, Saturation, Value) with mean value among all non-black pixels    
    """
    flattened = image.reshape(-1, image.shape[-1])
    HUE, SATURATION, VALUE = (0,1,2)
    hue_components = flattened[:, HUE]
    saturation_components = flattened[:, SATURATION]
    value_components = flattened[:, VALUE]
    mask = (hue_components != 0) | (saturation_components != 0) | (value_components != 0)
    HSV_mean = np.mean(flattened[mask, :], axis = 0)
    hue, saturation, value = HSV_mean[0], HSV_mean[1], HSV_mean[2]
    return hue, saturation, value


In [ ]:
#F31-F36 color features
F31_all = np.zeros(n_ALL)
F32_all = np.zeros(n_ALL)
F33_all = np.zeros(n_ALL)
F31_hem = np.zeros(n_hem)
F32_hem = np.zeros(n_hem)
F33_hem = np.zeros(n_hem)
F34_all = np.zeros(n_ALL)
F35_all = np.zeros(n_ALL)
F36_all = np.zeros(n_ALL)
F34_hem = np.zeros(n_hem)
F35_hem = np.zeros(n_hem)
F36_hem = np.zeros(n_hem)
for i in range(n_ALL):
    F31_all[i],F32_all[i],F33_all[i] = image_RGB_mean(ALL_0_color[i])
    ALL_HSV = image_HSV(ALL_0_color[i])
    F34_all[i],F35_all[i],F36_all[i] = image_HSV_mean(ALL_HSV)
for j in range(n_hem) :
    F31_hem[j],F32_hem[j],F33_hem[j] = image_RGB_mean(hem_0_color[j])
    hem_HSV = image_HSV(hem_0_color[j])
    F34_hem[j],F35_hem[j],F36_hem[j] = image_HSV_mean(hem_HSV)
    
F31_t = np.zeros(n_test)
F32_t = np.zeros(n_test)
F33_t = np.zeros(n_test)
F34_t = np.zeros(n_test)
F35_t = np.zeros(n_test)
F36_t = np.zeros(n_test)
for i in range(n_test):
    F31_t[i],F32_t[i],F33_t[i] = image_RGB_mean(test_color[i])
    test_HSV = image_HSV(test_color[i])
    F34_t[i],F35_t[i],F36_t[i] = image_HSV_mean(test_HSV)

In [ ]:
def intensity(image):
    """
    arg:
        image: grayscale square iamges
    returns: 
        mean: intensity mean of nonzero pixels
        var: intensity variance of nonzero pixels
    """
    shape = np.shape(image)
    image = image.reshape(shape[0],shape[1]*shape[2])
    mean = np.zeros((shape[0]),)
    var = np.zeros((shape[0]),)
    for i in range(shape[0]):
        mask = (image[i] != 0)
        mean[i] = np.mean(image[i,mask], axis=0)
        var[i] = np.var(image[i,mask], axis=0)
    return mean, var


In [ ]:
F37_all, F38_all = intensity(ALL_0_gray)
F37_hem, F38_hem = intensity(hem_0_gray)
F37_t, F38_t = intensity(test_gray)

In [ ]:
# combine all features for training data
ALL1 = np.stack((F1_all,F2_all,F3_all,F4_all,F5_all,F6_all,F7_all,F8_all,F9_all), axis=1)
ALL3 = np.stack((F31_all,F32_all,F33_all,F34_all,F35_all,F36_all,F37_all,F38_all), axis=1)
ALL = np.concatenate((ALL1,Haralick_all,Haar_wavelet_all,ALL3), axis=1)
hem1 = np.stack((F1_hem,F2_hem,F3_hem,F4_hem,F5_hem,F6_hem,F7_hem,F8_hem,F9_hem), axis=1)
hem3 = np.stack((F31_hem,F32_hem,F33_hem,F34_hem,F35_hem,F36_hem,F37_hem,F38_hem), axis=1)
hem = np.concatenate((hem1,Haralick_hem,Haar_wavelet_hem,hem3), axis=1)
print('Shape of ALL:', np.shape(ALL))
print('Shape of hem:', np.shape(hem))

# Combine ALL and hem 
raw_train_data = np.concatenate((ALL,hem), axis=0)

print('Shape of training data:', np.shape(raw_train_data))


In [ ]:
# Feature standardization
scaler = MinMaxScaler()
train_data = scaler.fit_transform(raw_train_data)
#data = StandardScaler().fit_transform(data)
train_data

# Conversion of data from numpy array to dataframe
train_data_df = pd.DataFrame.from_records(train_data)
train_data_df.columns = features
label_train = np.concatenate(([1]*n_ALL,[0]*n_hem), axis=0) 
train_data_df.insert(0,"Label",label_train,True)

# save train_data_df as a csv file
train_data_df.to_csv(r'C:\Users\1000j\CX4240+\ProjectALL\train_data.csv')
train_data_df

In [ ]:
# comnine all features for test data
test_data1 = np.stack((F1_t,F2_t,F3_t,F4_t,F5_t,F6_t,F7_t,F8_t,F9_t), axis=1)
test_data3 = np.stack((F31_t,F32_t,F33_t,F34_t,F35_t,F36_t,F37_t,F38_t), axis=1)
test_data = np.concatenate((test_data1,F10_F22_t,F23_F30_t,test_data3), axis=1)
print('Shape of test data:', np.shape(test_data))

# Feature standardization
test_data = scaler.transform(test_data)
test_data

test_data_df = pd.DataFrame.from_records(test_data)
test_data_df.columns = features

# load labels for test data 
label_test_df = pd.read_csv (r'C:\Users\1000j\CX4240+\Cell_Images\C-NMC_test_prelim_phase_data\C-NMC_test_prelim_phase_data_labels.csv')
print(label_test_df)
label_test = label_test_df['labels'].as_matrix()
test_data_df.insert(0,"Label",label_test_df['labels'],True)

# save test_data_df as a csv file
test_data_df.to_csv(r'C:\Users\1000j\CX4240+\ProjectALL\test_data.csv')
test_data_df